In [207]:
import os
import pandas as pd
import numpy as np
from string import punctuation
import glob
import nltk
import re
import math
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/diegohermosillo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/diegohermosillo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [208]:
punctuation = list(punctuation)
stop_words = stopwords.words('english')

In [209]:
def processing(text):
    # quitar espacios extra en los lados y pasar a minúsculas
    new_text = text.strip().lower()
    # quitar doble espacio
    new_text = new_text.replace('  ', '')
    new_text = new_text.replace("'", '')
    new_text = new_text.replace("-", '')
    new_text = re.sub("[^a-zA-Z\s]+", '', new_text)
    # separar texto en tokens
    tokens = word_tokenize(new_text)
    # guardar tokens que no estén en las stopwords ni sean signos de puntuación
    cleaned_tokens = [token for token in tokens if token not in stop_words 
                      and token not in punctuation and len(token) > 1]
    
    # stemmer
    ps = PorterStemmer()
    stem_tokens = [ps.stem(token) for token in cleaned_tokens]

    return stem_tokens

def read_processed_file(file):
    # Función que lee un archivo procesado
    f = open(file, 'r')
    text = f.read()
    f.close()
    new_text = text.strip()
    print(new_text.split('\n'))

def create_processed_file(tokens):
    # Función que guarda el texto procesado en otro archivo
    with open('01', 'w') as file:
        for t in tokens:
            file.write(f'{t}\n')

In [210]:
# if not os.path.exists('procesados'):
#     os.mkdir('procesados')

# definir directorio de archivos aún sin procesar
directory = 'sinprocesar'

# numero de archivos y lista de archivos ordenados
ordered_files = sorted(glob.glob(f'{directory}/*'))
num = len(ordered_files)


tf = {}
n = {}
# iterar sobre los archivos en ese directorio
for id, filename in enumerate(ordered_files):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    tokens = processing(text)
    
    for t in set(tokens):
        # guarda como llave el termino y como valor una lista de tamaño N y después se modifica el tf
        if t not in tf:
            tf[t] = {}
        tf[t][id] = 1 + math.log2(tokens.count(t))
        # guarda el conteo de las palabras
        n[t] = n.get(t,0) + tokens.count(t)

#del punctuation, ordered_files, directory, tokens
# A n sacar el idf = log(N/ni) donde
#           N : Número total de documentos
#           ni : número de documentos donde aparece (o sea el conteo)

idf = {k:math.log2(num/v) for (k,v) in n.items()}

In [211]:
tf_idf = {}
for k,v in idf.items():
    diccionario = tf.get(k)
    tf_idf[k] = {k1: v1 * v for k1, v1 in diccionario.items()}

In [212]:
from string import punctuation
punctuation = list(punctuation)


In [213]:
# definir directorio de archivos aún sin procesar
directory = 'sinprocesar'

# numero de archivos y lista de archivos ordenados
ordered_files = sorted(glob.glob(f'{directory}/*'))
num = len(ordered_files)

longitud = []

for id, filename in enumerate(ordered_files):
    valor = 0
    file = open(filename, 'r')
    text = file.read()
    file.close()
    tokens = processing(text)

    for t in set(tokens):
        valor += tf_idf[t][id]
        
    longitud.append(math.sqrt(math.pow(valor,2)))

In [214]:
def save_file(data, name):
    with open(f"{name}.pkl", 'wb') as fp:
        pickle.dump(data, fp)

def read_file(name):
    with open(f"{name}.pkl", 'rb') as fp:
        return pickle.load(fp)

In [215]:
# save dictionary to tf_idf, idf and longitud file
save_file(tf_idf, "tf_idf")
save_file(idf, "idf")
save_file(longitud, "longitud")

In [216]:
#Sección 3: Búsqueda de 1 palabra (dividiendo entre la longitud)
consulta = input("Please make the query (1 word only): ")
num_results=int(input("How many results would you like to see?"))
consulta_procesada=str(processing(consulta)) #El mismo pre procesamiento que se le hizo a los documentos se le aplica a la consulta
consulta_procesada = consulta_procesada.strip("[]'")
try:
    resultados_ranking = {i: tf_idf[consulta_procesada][i] / longitud[i] for i in tf_idf[consulta_procesada].keys()} #Las llaves son los documentos
    ranking_ordenado=sorted(resultados_ranking.items(), key=lambda item: item[1],reverse=True)[:num_results] #Aquí ya los ordena
    for key,value in ranking_ordenado:
        print(f"Document number {key} with a weight of {value}")
except KeyError:
    print("No results")


ValueError: invalid literal for int() with base 10: 's'

In [ ]:
#Búsquedas con más de 1 palabra con formato AND
#Sirve con queries como "how many galaxies"
consulta_and = input("Please make the query. This is an AND query.")
num_results_and=int(input("How many results would you like to see?"))
consulta_procesada_and = str(processing(consulta_and))  # Apply the same preprocessing as done for the documents to the query
terminos = consulta_procesada_and.strip("[]'").replace("'", "").split(", ")

# Encontrar documentos que tengan los términos (ya procesados) de la búsqueda
try:
    docs_comunes = set.intersection(*(set(tf[i].keys()) for i in terminos))
    resultados_and={}
    for index, doc_number in enumerate(docs_comunes):
        total_tfidf = sum(tf_idf[termino][doc_number] for termino in terminos) #Se suman los tf_idf de cada termino a nivel documento
        resultados_and[doc_number]=total_tfidf

    resultados_and_ordenados=sorted(resultados_and.items(),key=lambda item: item[1],reverse=True)[:num_results_and]
    for key, value in enumerate(resultados_and_ordenados):
        print(f"Document number {key} with a weight of {value}")

except KeyError:
    print("No results")



In [ ]:
#Consulta con comillas
#Hacer la búsqueda con black hole
consulta_comillas=input("Please make your query (between quotes " ")").strip('"').lower()
num_results_comillas=int(input("How many results would you like to see?"))
resultados_comillas={}
try:
    for id, filename in enumerate(ordered_files):
        file = open(filename, 'r')
        text = file.read().lower()
        file.close()
        if consulta_comillas in text:
            resultados_comillas[filename]=text.count(consulta_comillas) #Elegimos hacer el ranking únicamente contando cuántas veces aparece la palabra

    resultados_comillas_ordenados=sorted(resultados_comillas.items(),key=lambda item: item[1],reverse=True)[:num_results_comillas]
    print(resultados_comillas_ordenados)
except KeyError:
    print("No results")

[('sinprocesar/000309446800033', 12), ('sinprocesar/000286385600046', 10)]
